# Chapter 10 Textual Data

You may not naturally think of _text_, like an e-mail or a newspaper article, as data. But just as we might predict the price of a home or cluster wines into similar types, we might want to predict the sender of an e-mail or cluster articles into similar types. To leverage the machine learning techniques from Part II of this book, we will need to convert raw text into tabular form. This chapter introduces some principles for doing so.

# 10.1 Bag of Words and N-Grams

In data science, a unit of text is typically called a _document_, even though a document can be anything from a text message to a full-length novel.  A collection of documents is called a _corpus_. In this lesson, we will work with a corpus of Dr. Seuss books.

In [1]:
import pandas as pd
import requests

seuss_dir = "http://dlsun.github.io/pods/data/drseuss/"
seuss_files = [
    "green_eggs_and_ham.txt", "cat_in_the_hat.txt", "fox_in_socks.txt",
    "hop_on_pop.txt", "horton_hears_a_who.txt", "how_the_grinch_stole_christmas.txt",
    "oh_the_places_youll_go.txt", "one_fish_two_fish.txt"
]

docs_seuss = pd.Series()
for file in seuss_files:
    response = requests.get(seuss_dir + file, "r")
    docs_seuss[file[:-4]] = response.text

docs_seuss

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()


green_eggs_and_ham                I am Sam\n\nI am Sam\nSam I am\n\nThat Sam-I-a...
cat_in_the_hat                    The sun did not shine.\nIt was too wet to play...
fox_in_socks                      Fox\nSocks\nBox\nKnox\n\nKnox in box.\nFox in ...
hop_on_pop                        UP PUP Pup is up.\nCUP PUP Pup in cup.\nPUP CU...
horton_hears_a_who                On the fifteenth of May, in the jungle of Nool...
how_the_grinch_stole_christmas    Every Who\nDown in Whoville\nLiked Christmas a...
oh_the_places_youll_go            Congratulations!\nToday is your day.\nYou're o...
one_fish_two_fish                 One fish, two fish, red fish, blue fish,\nBlac...
dtype: object

Suppose, for example, that we want to determine which two Dr. Seuss books are most similar or cluster the books into several types. In order to leverage the methods that we learned in Parts I and II of this book, we have to convert these documents into tabular form. In this lesson, we focus on a particular representation of a document called the _bag of words_.

The _bag of words_ representation reduces a document to just the multiset of its words, ignoring grammar and word order. (A _multiset_ is like a set, except that elements are allowed to appear more than once.)

So, for example, the **bag of words** representation of "I am Sam. Sam I am." (the first two lines of _Green Eggs and Ham_) would be `{I, I, am, am, Sam, Sam}`. In Python, it is easiest to represent multisets using dictionaries, where the keys are the (unique) words and the values are the counts. So we would represent the above bag of words as `{"I": 2, "am": 2, "Sam": 2}`.

Let's convert the Dr. Seuss books to a bag of words representation. To do this, we will use the `Counter` object in the `collections` module of the Python standard library. First, let's see how the `Counter` works.

In [2]:
from collections import Counter
Counter(["I", "am", "Sam", "Sam", "I", "am"])

Counter({'I': 2, 'Sam': 2, 'am': 2})

It takes in a list and returns a dictionary of counts---in other words, the bag of words representation that we want. But to be able to use `Counter`, we have to first convert our document into a list of words. We can do this using the string methods in Pandas, such as `.str.split()`, which splits a string into a list based on some character (which, by default, is whitespace).

In [3]:
docs_seuss.str.split()

green_eggs_and_ham                [I, am, Sam, I, am, Sam, Sam, I, am, That, Sam...
cat_in_the_hat                    [The, sun, did, not, shine., It, was, too, wet...
fox_in_socks                      [Fox, Socks, Box, Knox, Knox, in, box., Fox, i...
hop_on_pop                        [UP, PUP, Pup, is, up., CUP, PUP, Pup, in, cup...
horton_hears_a_who                [On, the, fifteenth, of, May,, in, the, jungle...
how_the_grinch_stole_christmas    [Every, Who, Down, in, Whoville, Liked, Christ...
oh_the_places_youll_go            [Congratulations!, Today, is, your, day., You'...
one_fish_two_fish                 [One, fish,, two, fish,, red, fish,, blue, fis...
dtype: object

There are several problems with this approach:

- **It is case-sensitive.**  The words "PUP" and "Pup" in _Hop on Pop_ are technically different strings and will be treated as different words by the `Counter`.
- **There is punctuation.**  For example, in _One Fish, Two Fish_, the words "fish," and "fish." will be treated as separate words.

We can **normalize** the text for case by 

- converting all of the characters to lowercase, using the `.str.lower()` method
- stripping punctuation using a regular expression. The regular expression `[^\w\s]` tells Python to look for any pattern that is not (`^`) either an alphanumeric character (`\w`) or whitespace (`\s`). That is, it will detect any occurrence of punctuation. We will then use the `.str.replace()` method to replace all detected occurrences with whitespace, effectively removing all punctuation from the string.

By chaining these commands together, we obtain a list, to which we can apply the `Counter` to obtain the bag of words representation.

In [49]:
words = (
    docs_seuss.
    str.lower().
    str.replace("[^\w\s]", " ").
    str.split()
)

words

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


green_eggs_and_ham                [i, am, sam, i, am, sam, sam, i, am, that, sam...
cat_in_the_hat                    [the, sun, did, not, shine, it, was, too, wet,...
fox_in_socks                      [fox, socks, box, knox, knox, in, box, fox, in...
hop_on_pop                        [up, pup, pup, is, up, cup, pup, pup, in, cup,...
horton_hears_a_who                [on, the, fifteenth, of, may, in, the, jungle,...
how_the_grinch_stole_christmas    [every, who, down, in, whoville, liked, christ...
oh_the_places_youll_go            [congratulations, today, is, your, day, you, r...
one_fish_two_fish                 [one, fish, two, fish, red, fish, blue, fish, ...
dtype: object

In [50]:
words.apply(Counter)

green_eggs_and_ham                {'i': 84, 'am': 16, 'sam': 19, 'that': 3, 'do'...
cat_in_the_hat                    {'the': 97, 'sun': 2, 'did': 10, 'not': 41, 's...
fox_in_socks                      {'fox': 17, 'socks': 19, 'box': 7, 'knox': 17,...
hop_on_pop                        {'up': 6, 'pup': 8, 'is': 12, 'cup': 4, 'in': ...
horton_hears_a_who                {'on': 21, 'the': 97, 'fifteenth': 1, 'of': 39...
how_the_grinch_stole_christmas    {'every': 5, 'who': 18, 'down': 10, 'in': 17, ...
oh_the_places_youll_go            {'congratulations': 1, 'today': 2, 'is': 7, 'y...
one_fish_two_fish                 {'one': 14, 'fish': 12, 'two': 4, 'red': 2, 'b...
dtype: object

## N-Grams

The problem with the bag of words representation is that the ordering of the words is lost. For example, the following sentences have the exact same bag of words representation, but convey different meanings:

1. The dog bit her owner.
2. Her dog bit the owner.

The first sentence has only two actors (the dog and its owner), but the second sentence has three (a woman, her dog, and the owner of something). To better capture the _semantic_ meaning of these two documents, we can use **bigrams** instead of individual words. A **bigram** is simply a pair of consecutive words. The "bag of bigrams" of the two sentences above are quite different:

1. {"The dog", "dog bit", "bit her", "her owner"}
2. {"Her dog", "dog bit", "bit the", "the owner"}

They only share 1 bigram (out of 4) in common, despite sharing the same 5 words.

Let's get the bag of bigrams representation for the words above. To generate the bigrams from the list of words, we will use the `zip` function in Python, which takes in two lists and returns a single list of pairs (consisting of one element from each list):

In [6]:
list(zip([1, 2, 3], [4, 5, 6]))

[(1, 4), (2, 5), (3, 6)]

In [7]:
def get_bigrams(words):
    # We need to line up the words as follows:
    #   words[0], words[1]
    #   words[1], words[2]
    #       ... ,  ...
    # words[n-1], words[n]
    #   words[n]
    # The first list is longer, so the last element in the first list is ignored.
    return zip(words, words[1:])

words.apply(get_bigrams).apply(Counter)

green_eggs_and_ham                {('i', 'am'): 16, ('am', 'sam'): 2, ('sam', 'i...
cat_in_the_hat                    {('the', 'sun'): 2, ('sun', 'did'): 1, ('did',...
fox_in_socks                      {('fox', 'socks'): 1, ('socks', 'box'): 1, ('b...
hop_on_pop                        {('up', 'pup'): 1, ('pup', 'pup'): 2, ('pup', ...
horton_hears_a_who                {('on', 'the'): 5, ('the', 'fifteenth'): 1, ('...
how_the_grinch_stole_christmas    {('every', 'who'): 4, ('who', 'down'): 4, ('do...
oh_the_places_youll_go            {('congratulations', 'today'): 1, ('today', 'i...
one_fish_two_fish                 {('one', 'fish'): 1, ('fish', 'two'): 1, ('two...
dtype: object

Instead of taking 2 words at a time, we could take 3, 4, or, in general, $n$ words. 
A tuple of $n$ consecutive words is called an $n$-gram, and we can convert any document to a "bag of $n$-grams" representation. 

The larger $n$ is, the better the representation will capture the meaning of a document. But if $n$ is so large that hardly any $n$-gram occurs more than once, then we will not learn much from this representation.

# Exercises

1\. Read in the OKCupid data set (`https://dlsun.github.io/pods/data/okcupid.csv`). Convert the users' responses to `essay0` ("self summary") into a bag of words representation. What word appears most often?

(_Hint:_ Test your code on the first 100 users before testing it on the entire data set.)

In [18]:
cupid = pd.read_csv("https://dlsun.github.io/pods/data/okcupid.csv")
essay0 = cupid["essay0"].dropna()

In [58]:
essays = (
    essay0.
    str.lower().
    str.replace("[^\w\s]", " ").
    str.split()
)
wordlist = essays.sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [59]:
essay_dict = Counter(wordlist)
Counter.most_common(essay_dict)[0]

('i', 22674)

**The word "i" appears most.**

2\. Find the bag of trigrams representation for the Dr. Seuss books. How could you use the bag of trigrams to determine which book is the most linguistically diverse? Which book is it?

In [56]:
def get_trigrams(words):
    return zip(words, words[1:], words[2:])

bag_trigrams = words.apply(get_trigrams).apply(Counter)
bag_trigrams

green_eggs_and_ham                {('i', 'am', 'sam'): 2, ('am', 'sam', 'i'): 1,...
cat_in_the_hat                    {('the', 'sun', 'did'): 1, ('sun', 'did', 'not...
fox_in_socks                      {('fox', 'socks', 'box'): 1, ('socks', 'box', ...
hop_on_pop                        {('up', 'pup', 'pup'): 1, ('pup', 'pup', 'is')...
horton_hears_a_who                {('on', 'the', 'fifteenth'): 1, ('the', 'fifte...
how_the_grinch_stole_christmas    {('every', 'who', 'down'): 4, ('who', 'down', ...
oh_the_places_youll_go            {('congratulations', 'today', 'is'): 1, ('toda...
one_fish_two_fish                 {('one', 'fish', 'two'): 1, ('fish', 'two', 'f...
dtype: object

In [67]:
diverse = []
for book in bag_trigrams:
  diverse.append(sum(book.values()))
diverse

[802, 1625, 892, 389, 2109, 1401, 1002, 1310]

In [75]:
max_val = max(diverse)
max_ind = diverse.index(max_val)
max_ind

4

In [81]:
bag_trigrams.keys()[max_ind - 1]

'hop_on_pop'

**Hop on Pop is the most linguistically diverse Dr. Seuss book.**